# Notebook for å opprette sqlite-database og raadata-tabell som grunnlag for visualisering med EDITH

### OBS! Denne kjøringen henter ut som default alle data knyttet til ett delregister. Dersom man kun ønsker data for ett (av flere skjema) må det legges til en seleksjon i uttrekket fra DYNAREV

# Laster inn pakker

In [ ]:
import pandas as pd
import sqlite3
from sqlalchemy import Table, create_engine
from sqlalchemy.sql import select
from flask_sqlalchemy import SQLAlchemy
import getpass
import cx_Oracle
import json
import os

from models.models_delt import connect
## Connect-funksjonen ligger i den fila, kjører i stedet som funksjon

# Laster inn forhåndsdefinert config.json
#### Fila må være fylt ut med riktige parametre og ligge i samme mappe som denne notebook'en

In [ ]:
#print(os.getcwd())

with open("config.json") as config: # Laster in valg fra config.json
    config = json.load(config)

In [ ]:
conn, engine, db = connect()
#engine, db = connect()

# Kobler til dynarev / oracle
#### Skriv inn eget Oracle passord

In [ ]:
bruker = getpass.getuser()[:3]  # Henter brukernavn
db = cx_Oracle.connect(bruker,getpass.getpass(), "DB1P")  # Skriv inn Oracle Passord

### Laster inn informasjon om enhetene

#### Liste med alle delreg_nr

In [ ]:
liste = []
for i in config["perioder"]:
    liste = liste + [config["perioder"][i]["delreg"]]

In [ ]:
# Henter informasjon fra SFU, ikke nødvendig å endre noe her!
# _KAN_ legge til ytterligere informasjon fra SFU, f eks aggregater. I koden under lages aggregat for kommune og fylke (se Oppsett_config_og_database.ipynb)

info_foretak_dict = {}
for i in liste:
    info_foretak_dict[i] = pd.read_sql(f"SELECT NAVN, ORGNR, ORG_FORM, NACE1,ENHETS_ID, ENHETS_TYPE,F_KOMMUNENR AS KOMMUNE, SUBSTR(F_KOMMUNENR,1,2) AS FYLKE FROM dynarev.vw_enhet_i_delreg WHERE DELREG_NR = {i}", con = db)
    info_foretak_dict[i]["orgnrNavn"] = info_foretak_dict[i]["ORGNR"]+str(": ")+info_foretak_dict[i]["NAVN"] # Lager en ny kolonne som heter OrgnrNAVN
#print(info_foretak_dict)

### Laster inn skjemadata for alle perioder

##### Eks på ekstra betingelse ved seleksjon fra vw_skjema_data: 
f"SELECT ENHETS_ID, FELT_ID, FELT_VERDI FROM DYNAREV.VW_SKJEMA_DATA WHERE DELREG_NR = {i} AND AKTIV=1 <b>AND skjema='RA-xxxx'</b>"


In [ ]:
# Kan endre navn fra FELT_ID (fra Dynarev) til noe man ønsker selv. Default er satt til VARIABEL. 
# Kan endres av ansvarlig hvis behov, men da må det også endres i models- og templates-variabler

data_skjema_dict = {}
for i in liste:
    data_skjema_dict[i] = pd.read_sql(f"SELECT ENHETS_ID,RAD_NR,LOPENR,SKJEMA, FELT_ID as VARIABEL, FELT_VERDI, DELREG_NR FROM DYNAREV.VW_SKJEMA_DATA WHERE DELREG_NR = {i} AND AKTIV=1 ", con = db)

In [ ]:
for i in config["perioder"]:
    data_skjema_dict[config["perioder"][i]["delreg"]][config["perioder"][i]["periode"]] = data_skjema_dict[config["perioder"][i]["delreg"]]['FELT_VERDI']
    data_skjema_dict[config["perioder"][i]["delreg"]].drop(['FELT_VERDI'], axis = 1, inplace = True)
#print(data_skjema_dict)

### Sammenkobler skjemadata og informasjon om foretak

In [ ]:
if len(data_skjema_dict) == 1:
    data_komplett = data_skjema_dict[config["perioder"]["t"]["delreg"]]
elif len(data_skjema_dict) > 1:
    data_komplett = data_skjema_dict[config["perioder"]["t"]["delreg"]]
    config["perioder"].pop("t")
    for i in config["perioder"]:
        data_komplett = pd.merge(data_komplett, data_skjema_dict[config["perioder"][i]["delreg"]][[config["perioder"][i]["periode"], "ENHETS_ID", "VARIABEL"]], on = ['ENHETS_ID', "VARIABEL"], how = 'left')
        #print(data_komplett.head())

In [ ]:
#print(os.getcwd())

with open("config.json") as config: # Laster in valg fra config.json
    config = json.load(config)

In [ ]:
# Rengjør dataene for data_komplett, som er den nye dataframen med informasjon om skjema

data_komplett = data_komplett.dropna(subset=[config['perioder']['t']['periode']]).drop_duplicates().reset_index(drop=True)
#print(data_komplett.head())

In [ ]:
# Kobler på sammen informasjon om foretak og skjema med en venstrekobling til data_komplett. ENHETS_ID er koblingsnøkkel. 

data_komplett = pd.merge(data_komplett, info_foretak_dict[config["perioder"]["t"]["delreg"]], on = ['ENHETS_ID'], how = 'left').reset_index(drop=True)
#data_komplett.head()

In [ ]:
# Kobler på flere aggregater

aggregater = {
    "Stratum": {
        "01": "Mini",
        "02": "Små",
        "03": "Mellomstore",
        "04": "Store",
        "05": "Mega",
        "98": "Med dyr, uten areal",
        "99": "Veksthus"
    },
    "Driftsform": {
        "01": "Korn og oljevekster",
        "02": "Øvrige jordbruksvekster",
        "03": "Hagebruksvekster",
        "04": "Storfe mjølkeproduksjon",
        "05": "Storfe kjøttproduksjon",
        "06": "Storfe mjølk- og kjøttproduksjon",
        "07": "Sau",
        "08": "Øvrige grovfôretende dyr og blandede produksjoner",
        "09": "Svin og fjørfe"
    }
}

df_aggregater = pd.read_sas(
    "/ssb/stamme02/jordbruk/sblbrund/wk8/lu_utvalg.sas7bdat",
    format = "sas7bdat",
    encoding="latin-1"
)

df_aggregater["Stratum"] = df_aggregater["STRATUM"].str[0:2]

df_aggregater["Driftsform"] = df_aggregater["STRATUM"].str[3:5]

for i in aggregater:
    df_aggregater[i] = df_aggregater[i].replace(aggregater[i])

df_aggregater = df_aggregater[["ORGNR"] + list(aggregater.keys())]

data_komplett = pd.merge(data_komplett, df_aggregater, on = "ORGNR")

In [ ]:
# data_komplett = data_komplett.rename(columns = {'FELT_ID': 'Variabel'})
data_komplett.to_sql(f'{config["tabeller"]["raadata"]}', con=engine, if_exists="replace",)

### Kontrollkjøring 

In [ ]:
#Kontrollkjøring på at tabell(er) er opprettet
tabeller = pd.read_sql("SELECT name FROM sqlite_master  WHERE type='table'", con=engine)

# Skriver ut hvilke tabeller databasen inneholder
tabeller



In [ ]:
# Kontrollkjøring for sjekk av innhold i raadata-tabell
grunnlag = pd.read_sql("SELECT * FROM raadata", con=engine)
#grunnlag.head(10)